In [46]:
from summit.strategies import TSEMO
from summit.domain import Domain, DomainError
from summit.utils.multiobjective import pareto_efficient, HvI
from summit.utils.optimizers import NSGAII
from summit.utils.models import ModelGroup, GPyModel
from GPy.kern import Exponential
from summit.utils.dataset import DataSet
from summit.benchmarks import DTLZ2
import warnings
import time
from fastprogress.fastprogress import master_bar, progress_bar
from summit.utils.multiobjective import pareto_efficient, HvI

In [39]:
errors = 0
num_inputs=6
num_objectives=2
lab = DTLZ2(num_inputs=num_inputs, 
            num_objectives=num_objectives)
models = {f'y_{i}': GPyModel(Exponential(input_dim=num_inputs,ARD=True))
          for i in range(num_objectives)}

# Initial suggestions
warnings.filterwarnings("ignore", category=RuntimeWarning)
tsemo = TSEMO(lab.domain, models=models, random_rate=0.00)
initial_experiments = tsemo.suggest_experiments(5*num_inputs)

# Run initial experiments
results = lab.run_experiments(initial_experiments)

Run one iteration of tsemo step by step

In [17]:
inputs, outputs = tsemo.transform.transform_inputs_outputs(results)

In [19]:
%timeit tsemo.models.fit(inputs, outputs, spectral_sample=False)

3.38 s ± 388 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%timeit tsemo.models['y_0'].spectral_sample(inputs, outputs, n_spectral_points=1500)

2.76 s ± 256 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%timeit tsemo.models['y_1'].spectral_sample(inputs, outputs, n_spectral_points=1500)

2.77 s ± 194 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%timeit tsemo.optimizer.optimize(tsemo.models, use_spectral_sample=True, pop_size=100, iterations=100)

265 ms ± 20.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Let's run with more data.

## More Data

In [29]:
errors = 0
num_inputs=6
num_objectives=2
lab = DTLZ2(num_inputs=num_inputs, 
            num_objectives=num_objectives)
models = {f'y_{i}': GPyModel(Exponential(input_dim=num_inputs,ARD=True))
          for i in range(num_objectives)}

# Initial suggestions
warnings.filterwarnings("ignore", category=RuntimeWarning)
tsemo = TSEMO(lab.domain, models=models, random_rate=0.00)
initial_experiments = tsemo.suggest_experiments(100)

# Run initial experiments
results = lab.run_experiments(initial_experiments)

In [30]:
inputs, outputs = tsemo.transform.transform_inputs_outputs(results)

In [31]:
%timeit tsemo.models.fit(inputs, outputs, spectral_sample=False)

7.1 s ± 583 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
%timeit tsemo.models['y_0'].spectral_sample(inputs, outputs, n_spectral_points=1500)

3.15 s ± 386 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%timeit tsemo.models['y_1'].spectral_sample(inputs, outputs, n_spectral_points=1500)

2.92 s ± 350 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%timeit tsemo.optimizer.optimize(tsemo.models, use_spectral_sample=True, pop_size=100, iterations=100)

484 ms ± 158 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
internal_res =tsemo.optimizer.optimize(tsemo.models, use_spectral_sample=True, pop_size=100, iterations=100)
hv_imp, indices = tsemo.select_max_hvi(
                outputs, internal_res.fun, 1
            )


It definitely does not make sense that that the algorithm is taking 2 days to run our server.

## Just try running it.

In [44]:
errors = 0
num_inputs=6
num_objectives=2
lab = DTLZ2(num_inputs=num_inputs, 
            num_objectives=num_objectives)
models = {f'y_{i}': GPyModel(Exponential(input_dim=num_inputs,ARD=True))
          for i in range(num_objectives)}

# Initial suggestions
warnings.filterwarnings("ignore", category=RuntimeWarning)
tsemo = TSEMO(lab.domain, models=models, random_rate=0.00)
initial_experiments = tsemo.suggest_experiments(5*num_inputs)

# Run initial experiments
experiments = lab.run_experiments(initial_experiments)

In [45]:
for i in progress_bar(range(100)):
    # Run experiments
    experiments = lab.run_experiments(experiments)

    # Get suggestions
    experiments = tsemo.suggest_experiments(1, experiments,
                                            use_spectral_sample=True, 
                                            pop_size=100, 
                                            iterations=100)

I figured out the issue was I was putting back in all the data at every iteration according to the old TSEMO API instead of just the most recent experiments

In [47]:
y_pareto, _ = pareto_efficient(lab.data[['y_0', 'y_1']].to_numpy(),
                               maximize=False)  
hv = HvI.hypervolume(y_pareto, [11,11])

In [48]:
hv

119.53606798055976

## More Spectral Points

In [50]:
errors = 0
num_inputs=6
num_objectives=2
lab = DTLZ2(num_inputs=num_inputs, 
            num_objectives=num_objectives)
models = {f'y_{i}': GPyModel(Exponential(input_dim=num_inputs,ARD=True))
          for i in range(num_objectives)}

# Initial suggestions
warnings.filterwarnings("ignore", category=RuntimeWarning)
tsemo = TSEMO(lab.domain, models=models, random_rate=0.00)
initial_experiments = tsemo.suggest_experiments(5*num_inputs)

# Run initial experiments
experiments = lab.run_experiments(initial_experiments)

In [ ]:
for i in progress_bar(range(100)):
    # Run experiments
    experiments = lab.run_experiments(experiments)

    # Get suggestions
    experiments = tsemo.suggest_experiments(1, experiments,
                                            use_spectral_sample=True,
                                            n_spectral_points=4000,
                                            pop_size=100, 
                                            iterations=100)